# Conversational Chatbot with Memory

## Imports

In [1]:
# System
import os
os.environ['USER_AGENT'] = 'JimYin88'

# LLM Models
from langchain_openai import ChatOpenAI
from langchain_ollama import OllamaLLM
from langchain_ollama import ChatOllama

# Templates
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain import hub


# Document Loaders
from langchain_community.document_loaders import Docx2txtLoader
from langchain_community.document_loaders import WebBaseLoader
# from langchain.document_loaders import BSHTMLLoader
# from langchain_community.document_loaders import UnstructuredRTFLoader

# Document Splitters
from langchain.text_splitter import TokenTextSplitter
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# Embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
# from langchain_openai import OpenAIEmbeddings

# Vector Stores
from langchain_chroma import Chroma
from langchain_community.vectorstores.faiss import FAISS

# LangChain Chains
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

# OutputParsers
from langchain.schema.output_parser import StrOutputParser

# Gradio
import gradio as gr

# Tools
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# History Memory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

## Load Environment Variables

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## Search Tool

In [3]:
search = TavilySearchResults(max_results=2)

## Retriever

In [4]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
splitter = TokenTextSplitter(chunk_size=200, chunk_overlap=20)
splitdocs = splitter.split_documents(docs)
embedding_function = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector_store = FAISS.from_documents(documents=splitdocs, embedding=embedding_function)
retriever = vector_store.as_retriever()

In [5]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [6]:
tools = [search, retriever_tool]

## Instantiate LLM model

### Using ChatGPT 4o-mini

In [7]:
chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                        max_completion_tokens=1028,
                        temperature=0.2)

In [8]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

## Create agent

In [9]:
agent = create_tool_calling_agent(llm=chat_model, 
                                  tools=tools, 
                                  prompt=prompt)

In [10]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

## Define Chatbot function

In [11]:
history = []

In [12]:
def chat(message, history):
    
    # messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = agent_executor.stream({"input": message, "chat_history": history})

    response = ""
    for chunk in stream:
        if 'output' in chunk:
            response += chunk['output'] or ''
        else:
            continue
        yield response

## Gradio Interface

In [13]:
chatbot = gr.ChatInterface(fn=chat, type="messages")

In [14]:
chatbot.launch(share=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [15]:
chatbot.close()

Closing server running on port: 7860
